The intention of this script is to first isolate key words and phrases common to events in the PostgreSQL database. Then to use these words to search for similar public Facebook events. The attendees for these events are collated into  the specific format requried for Facebook custom audience generator.

In [14]:
from facebook import GraphAPI
import requests
import urllib3
import numpy as np

#has to be updated from Facebook Graph API explorer every so often.
token = 'XXXXXX'

import pandas as pd
import psycopg2
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
#contains punctuation library that can be checked against.
import string
#checking if words in English dictionary
#import enchant 
#for ordering dictionary by values
import operator 
#for counting phrases and sentences
import requests
import json

from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk import FreqDist
import time
import pycountry 
conn = psycopg2.connect("dbname=XXXX user=XXXX host=XXXX port=XXXX password=XXXXX")

graph = GraphAPI(token)

Access the database and select event information (name, description, location etc.)

In [15]:
cur = conn.cursor()
cur.execute("""SELECT
  events.name,
  events.description,
  sports.name,
  venues.address,
  venues.coordinates
FROM events
  INNER JOIN sports ON events.sport_id = sports.id
  INNER JOIN events_venues ON events.id = events_venues.event_id
  INNER JOIN venues ON events_venues.venue_id = venues.id
""")
out = cur.fetchall()
cur.close()

cur_two = conn.cursor()
cur_two.execute("""SELECT
  facebook_connections.facebook_id
FROM
  facebook_connections
""")
out_two = cur_two.fetchall()
cur_two.close()
conn.close()

some simple functions for use later.

In [16]:
def getKey(item):
    return item[1]

#function that takes token and returns evenly spaced string (Token in tuple form ("word","word2")... so need to remove "()" and ","
def stringify(token):
    if len(token) == 1:
        result = token[0]
    else:
        result = " "
        for i in token:
            result += i
            result += " "
    return result 

This goes through the database, correctly formats all text associated with the events into a way that allows it to be analysed for key words and phrases.

In [17]:
sport_cohorts = defaultdict(list)
for element in out:
    sport_cohorts[element[2]].append((element[0],element[1]))
sport_cohorts_strings = defaultdict(list)
for i in sport_cohorts.keys():
    for j in sport_cohorts[i]:
            sport_cohorts_strings[i].append(j[0])        
            if j[1] != None:
                sport_cohorts_strings[i].append(j[1])
sport_cohorts_punc = defaultdict(list)
for i in sport_cohorts_strings.keys():
    for j in sport_cohorts_strings[i]:
        for c in string.punctuation:
            j = j.replace(c,"")
        sport_cohorts_punc[i].append(j)
full_strings = {}
for i in sport_cohorts_punc.keys():
    full_strings[i] = " "
    for j in sport_cohorts_punc[i]:
        full_strings[i] += j
        full_strings[i] += " "
big_dictionary = defaultdict(dict)
for i in full_strings.keys():
    piece = full_strings[i]
    tokens = word_tokenize(piece)
    counts = dict()
    for size in 1, 2, 3, 4, 5:
        counts[size] = FreqDist(ngrams(tokens, size))
    big_dictionary[i] = counts 

Look through Facebook Graph API to search for public Facebook events featuring the key words or phrases being searched for.

In [22]:
def get_information(phrase):
    list_IDs = []
    link = "https://graph.facebook.com/search?q={}&type=event&access_token={}".format(phrase,token)
    r = requests.get(link)
    result = json.loads(r.text)
    if u'error' in result.keys():
        print "BLOCKED AGAIN"
        result = {}
    if u'data' in result.keys():
        for event in result[u'data']:

            if u'id' in event.keys():
                if u'place' in event.keys():
                    if u'location' in event[u'place']:
                        if u'city' in event[u'place'][u'location'].keys():
                            city = event[u'place'][u'location'][u'city'].encode('utf8')
                        else:
                            city = " "
                        if u'state' in event[u'place'][u'location'].keys():
                            state = event[u'place'][u'location'][u'state'].encode('utf8')
                        else:
                            state = " " 
                        if u'zip' in event[u'place'][u'location'].keys():
                            zip_code = event[u'place'][u'location'][u'zip'].encode('utf8')
                        else:
                            zip_code = " "
                     
                        if u'country' in event[u'place'][u'location'].keys():
                        
                            mapping = {country.name: country.alpha_2 for country in pycountry.countries}
                            country = mapping.get(event[u'place'][u'location'][u'country'])
                            if country != None:
                                country = country.encode('utf8')
                            else: 
                                country = " "
                        else:
                            country = " "
                        list_IDs.append((result[u'data'][result[u'data'].index(event)][u'id'],city,state,zip_code,country))
                               
    list_attendees_IDs = []
    for IDs in list_IDs:
        ID = IDs[0]
        city = IDs[1]
        state = IDs[2]
        zip_code = IDs[3]
        country = IDs[4]
        link_two = "https://graph.facebook.com/{}?fields=attending&access_token={}".format(ID,token)
        p = requests.get(link_two)
        result_p = json.loads(p.text)
        
        if u'attending' in result_p.keys():
            for person in result_p[u'attending'][u'data']:
                
                first_name = person[u'name'].encode('utf8').split(" ",1)[0]
                if len(person[u'name'].encode('utf8').split(" ",1)) !=1:
                    second_name = person[u'name'].encode('utf8').split(" ",1)[1]
                else:
                    second_name = " "
                list_attendees_IDs.append((first_name,second_name,city,state,zip_code,country))
            
            counter = 1
            
            while counter <25:
                if u'attending' in result_p.keys():
                    if u'paging' in result_p[u'attending'].keys():
                        if u'next' in result_p[u'attending'][u'paging'].keys():
                            q = requests.get(result_p[u'attending'][u'paging'][u'next'])
                            result_q = json.loads(q.text)
                            if u'data' in result_q.keys():
                                for person in result_q[u'data']:
                                    first_name = person[u'name'].encode('utf8').split(" ",1)[0]
                                    if len(person[u'name'].encode('utf8').split(" ",1)) !=1:
                                        second_name = person[u'name'].encode('utf8').split(" ",1)[1]
                                    else:
                                        second_name = " "
                                    list_attendees_IDs.append((first_name,second_name,city,state,zip_code,country))
                                result_p = result_q
                                counter +=1
                            else:
                                counter = 1000
                                result_p = {}
                        else:
                            counter = 1000
                            result_p = {}
                    else:
                        counter = 1000
                        result_p = {}
                else:
                    if u'paging' in result_p.keys():
                        if u'next' in result_p[u'paging'].keys():
                            q = requests.get(result_p[u'paging'][u'next'])
                            result_q = json.loads(q.text)
                            if u'data' in result_q.keys():
                                for person in result_q[u'data']:
                                    first_name = person[u'name'].encode('utf8').split(" ",1)[0]
                                    if len(person[u'name'].encode('utf8').split(" ",1)) !=1:
                                        second_name = person[u'name'].encode('utf8').split(" ",1)[1]
                                    else:
                                        second_name = " "
                                    list_attendees_IDs.append((first_name,second_name,city,state,zip_code,country))
                                
                                    result_p = result_q
                                    counter +=1
                            else:
                                counter = 1000
                                result_p = {}
                        else:
                            counter = 1000
                            result_p = {}
                    else:
                        counter = 1000
                        result_p = {}
    return list_attendees_IDs                    
    
Sport = "Football"
Looking_at = 3        #number of words in phrases being searched for 

print "Number of phrases, ", len(big_dictionary[Sport][Looking_at].keys())
big_list_IDs = []
counter = 0
start = time.time()
for phrases in big_dictionary[Sport][Looking_at].keys()[1301:1500]:
    for ID in get_information(phrases):
        big_list_IDs.append(ID)
    counter +=1
    if counter in [10,20,30,40,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950]:
        print counter
        end = time.time()
        print (end-start)
print "Number of Id's ",len(big_list_IDs)
print "done"

Number of phrases,  34163
10
9.33487010002
20
28.5677599907
30
32.6485729218
40
38.400947094
50
52.9252650738
100
131.686681986
150
245.492316961
Number of Id's  9713
done


The script above generates a list of entires for each person found to be attending one of the public events in question. This script converts this list into a .csv for ease and time saving.

In [23]:
import csv
d = big_list_IDs
with open("1500.csv","wb") as outfile:
    writer = csv.writer(outfile)
    writer.writerow([i for i,j,k,v,m,n in d])
    writer.writerow([j for i,j,k,v,m,n in d])
    writer.writerow([k for i,j,k,v,m,n in d])
    writer.writerow([v for i,j,k,v,m,n in d])
    writer.writerow([m for i,j,k,v,m,n in d])
    writer.writerow([n for i,j,k,v,m,n in d])
    

This script takes all of the .csv files created throughout the above process (above process has to be done several times for different portions of the data set as it is too computationally intensive otherwise).
This combines all of this information into one pandas dataframe, formats it into the exact requirements of Facebook custom audience creator. Then converts it all back to a final .csv file for submission.

In [24]:
import pandas as pd

list_csv = ['400.csv','600.csv','800.csv','1100.csv','1300.csv','1500.csv']

list_final = []

df_f = pd.DataFrame()
for link in list_csv:
    df = pd.read_csv(link)
    df_new = pd.concat([df_f,df],axis=1)
    df_f = df_new

df_transposed = df_f.T


#removed state for the moment as it means we lose a lot of data.

del df_transposed[2]

df_transposed = df_transposed[df_transposed[0] != " "]
df_transposed = df_transposed[df_transposed[1] != " "]
#df_transposed = df_transposed[df_transposed[2] != " "]
df_transposed = df_transposed[df_transposed[3] != " "]
df_transposed = df_transposed[df_transposed[4] != " "]

df_transposed.columns = ["ln","ct","zip","country"]
df_transposed.index.names = ["fn"]
print df_transposed.head()

df_transposed.to_csv("name.csv")

                 ln           ct      zip country
fn                                               
Paul        Drabick   Pittsburgh    15226      US
Maria    Ofi Olvera  San Antonio    78221      US
Stephen     Chapman      Calgary  T2E 8P1      CA
Connor        Kopko      Boonton    07005      US
Clayton     Carroll      Boonton    07005      US
